In [ ]:
import pyodbc
import pandas as pd
import os

# List of CSV files and their corresponding table names
csv_files = [
    ("customers.csv", "customers"),
    ("geolocation.csv", "geolocation"),
    ("order_items.csv", "order_items"),
    ("orders.csv", "orders"),
    ("payments.csv", "payments"),
    ("products.csv", "products"),
    ("sellers.csv", "sellers")
]

# Step 1: Connect to SQL Server
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=GUNJA\\SQLEXPRESS;'
    'DATABASE=e_commeres;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = r"C:\Users\sumit\OneDrive\Documents\New folder\E-Commers"

# Function to map pandas dtypes to SQL Server types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BIT'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'NVARCHAR(MAX)'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)

    # Convert columns that look like dates
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=False)
            except Exception as e:
                print(f"Warning: Could not convert {col} to datetime - {e}")

    df = df.where(pd.notnull(df), None)  # Replace NaN with None
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    print(f"\nProcessing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Create table if it doesn't exist
    columns = ', '.join([f'[{col}] {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f"""
    IF OBJECT_ID(N'{table_name}', N'U') IS NULL
    BEGIN
        CREATE TABLE [{table_name}] ({columns})
    END
    """
    cursor.execute(create_table_query)

    # Insert data
    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        placeholders = ', '.join(['?'] * len(row))
        sql = f"INSERT INTO [{table_name}] ({', '.join([f'[{col}]' for col in df.columns])}) VALUES ({placeholders})"
        cursor.execute(sql, values)

    conn.commit()

conn.close()
